In [1]:
# parameters
path_video_input = "../data/inputs/kth_dataset/handclapping/person03_handclapping_d2_uncomp.avi"
path_video_output = "../data/keypoints/on_videos/kth_dataset/handclapping/person03_handclapping_d2_uncomp_out.mp4"

In [2]:
#new_path = path_video_input.replace(("inputs","keypoints/on_videos"))
from pathlib import Path
assert Path(path_video_input).exists()
Path(path_video_output).parent.mkdir(exist_ok=True,parents=True)

In [3]:
#new_path = path_video_input.replace(("inputs","keypoints/on_videos"))
from pathlib import Path
suffix = Path(path_video_input).suffix
path_keypoint_output = path_video_input.replace("inputs","keypoints/as_arrays").replace(
    suffix,"_out"+".npz")
Path(path_keypoint_output).parent.mkdir(exist_ok=True,parents=True)

In [4]:
# charging gpu if available
import numpy as np,torch
device = "cuda" if torch.cuda.is_available() else "cpu"


In [5]:
#loading model
from models import keypoint_detector_mod
keypoint_detector = keypoint_detector_mod.KeyPointDetector(device,0.0)

In [7]:
already_processed = Path(path_video_output).exists() and  Path(path_keypoint_output).exists()
if not(already_processed):
    import video
    import time
    start = time.time()
    video_handler = video.VideoHandler(path_video_input,truncate=None,batch_size=10)
    kpt_detection = keypoint_detector.predict_on_whole_video(video_handler)
    video_handler.write(path_video_output)
    stop = time.time()
    print(stop-start)
else:
    print(f"skipping {str(path_video_input)} as it was already processed")

skipping ../data/inputs/kth_dataset/handclapping/person03_handclapping_d2_uncomp.avi as it was already processed


In [8]:
if not(already_processed):
    import keypoints_io
    keypoints_io.save(path_keypoint_output,kpt_detection)

In [9]:
if not(already_processed):
    for i in range(2):
        assert all([np.all(el1==el2) 
            for el1,el2 in zip(keypoints_io.load(path_keypoint_output)[i], kpt_detection[i])])

In [ ]:
if not(already_processed):
    del kpt_detection,video_handler
del keypoint_detector